In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 27 (delta 10), reused 23 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 6.89 KiB | 3.45 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/kaggle/working/agnews-classifier


In [10]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=6,
    lora_alpha=18,
    lora_dropout=0.1,
    bias='lora_only',
    target_modules=['query','key','value'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 953,092 || all params: 125,574,152 || trainable%: 0.7590


In [11]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [12]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='rmsprop',
    learning_rate=5e-5,
    lr_scheduler_type='cosine',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,0.733500,0.325711,0.893750
200,0.353900,0.324712,0.893750
300,0.330800,0.349773,0.885938
400,0.304500,0.324162,0.900000
500,0.300100,0.328883,0.896875
600,0.287300,0.311106,0.910937
700,0.294200,0.327940,0.901563
800,0.296900,0.306608,0.907813
900,0.289800,0.311441,0.907813
1000,0.293800,0.302497,0.907813


In [13]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2000'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:59<00:00,  4.21it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2000.csv


In [14]:
import os
import pandas as pd

# Define paths
csv_path = "/kaggle/working/saved_models/checkpoint-2000/log_history.csv"
output_dir = "/kaggle/working/processed_data"
output_csv = os.path.join(output_dir, "processed_log_history.csv")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the csv file
df = pd.read_csv(csv_path)

# Select desired columns and reorder the dataframe
desired_order = [
    "step",         
    "loss",         
    "eval_loss",   
    "accuracy",     
    "eval_accuracy",
    "learning_rate",
    "epoch"         
]
df = df[desired_order]

# Rename columns for uniformity
df.rename(columns={
    "step": "Step",
    "loss": "Train Loss",
    "eval_loss": "Test Loss",
    "accuracy": "Train Acc",
    "eval_accuracy": "Test Acc",
    "learning_rate": "Learning Rate",
    "epoch": "Epochs"
}, inplace=True)

# Adding loss spread and loss ratio columns
df["loss spread"] = df["Train Loss"] - df["Test Loss"]
df["loss ratio"] = df["Train Loss"] / df["Test Loss"]

# Adding acc spread and acc ratio columns
df["Acc spread"] = df["Test Acc"] - df["Train Acc"]
df["Acc ratio"] = df["Test Acc"] / df["Train Acc"]

# Export the DataFrame as a csv file
df.to_csv(output_csv, index=False)

print(f"Data exported to {output_csv}")

df

Data exported to /kaggle/working/processed_data/processed_log_history.csv


,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,loss spread,loss ratio,Acc spread,Acc ratio
0,100,0.7335,0.325711,0.731250,0.893750,4.969221e-05,0.013405,0.407789,2.251999,0.162500,1.222222
1,200,0.3539,0.324712,0.889286,0.893750,4.877641e-05,0.026810,0.029188,1.089888,0.004464,1.005020
2,300,0.3308,0.349773,0.886161,0.885938,4.727516e-05,0.040214,-0.018973,0.945756,-0.000223,0.999748
3,400,0.3045,0.324162,0.893750,0.900000,4.522542e-05,0.053619,-0.019662,0.939344,0.006250,1.006993
4,500,0.3001,0.328883,0.900446,0.896875,4.267767e-05,0.067024,-0.028783,0.912483,-0.003571,0.996034
5,600,0.2873,0.311106,0.904464,0.910937,3.969463e-05,0.080429,-0.023806,0.923480,0.006473,1.007157
6,700,0.2942,0.327940,0.907589,0.901563,3.634976e-05,0.093834,-0.033740,0.897116,-0.006027,0.993360
7,800,0.2969,0.306608,0.898661,0.907813,3.272542e-05,0.107239,-0.009708,0.968338,0.009152,1.010184
8,900,0.2898,0.311441,0.904464,0.907813,2.891086e-05,0.120643,-0.021641,0.930512,0.003348,1.003702
9,1000,0.2938,0.302497,0.900893,0.907813,2.500000e-05,0.134048,-0.008697,0.971249,0.006920,1.007681
